<a href="https://colab.research.google.com/github/fznnfitrah/algorithm-bm25-tki/blob/main/Bm25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from rank_bm25 import BM25Okapi
import re



1. Membaca data dari file CSV bernama tmdb_top_rated_movies.csv dan menyimpannya ke dalam sebuah DataFrame Pandas bernama df.

In [ ]:
df = pd.read_csv('tmdb_top_rated_movies.csv')

2. memproses judul yang telah didapat sebelumnya
3. Membuat sebuah daftar (list) yang berisi judul-judul film yang telah melalui proses preprocessing. Daftar ini akan menjadi "korpus" dokumen yang akan diindeks oleh algoritma BM25.

In [ ]:
# 2. Preprocessing judul film
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 3. Buat korpus judul yang sudah dipreprocess
df['processed_title'] = df['title'].apply(preprocess_text)
corpus = df['processed_title'].dropna().tolist()


4. Proses memecah teks menjadi unit-unit yang lebih kecil, biasanya kata-kata atau frasa. BM25 adalah algoritma yang bekerja pada tingkat kata. Oleh karena itu, setiap judul film yang telah diproses perlu dipecah menjadi kata-kata individual. Untuk setiap doc (yang merupakan sebuah judul film yang sudah diproses) dalam corpus, metode .split() digunakan. Secara default, .split() akan memecah string berdasarkan spasi, menghasilkan sebuah list kata-kata. Hasilnya adalah sebuah list dari list, di mana setiap list di dalamnya berisi kata-kata dari sebuah judul film.

In [ ]:
# 4. Tokenisasi judul film
tokenized_corpus = [doc.split() for doc in corpus]

5. Di baris ini, kita membuat sebuah instan objek dari kelas BM25Okapi yang telah kita impor sebelumnya. Objek ini akan kita gunakan untuk melakukan perhitungan skor relevansi. Saat kita membuat objek BM25Okapi, kita perlu memberikan korpus dokumen yang akan diindeks. Proses inisialisasi ini memungkinkan objek bm25 untuk mempelajari karakteristik kata-kata dalam korpus, seperti frekuensi kata dan invers dokumen frekuensi (IDF), yang akan digunakan nanti dalam perhitungan skor relevans

In [ ]:
# 5. Inisialisasi BM25
bm25 = BM25Okapi(tokenized_corpus)

6. Mendefinisikan sebuah fungsi bernama search_movies yang menerima query pencarian dan jumlah hasil teratas yang ingin dikembalikan (top_n, dengan nilai default 5). Fungsi ini akan mencari film-film yang paling relevan dengan query tersebut berdasarkan algoritma BM25.

In [ ]:

# 6. Fungsi pencarian
def search_movies(query_text, top_n=5):
    # proses query
    processed_query = preprocess_text(query_text)
    tokenized_query = processed_query.split()

    # Dapatkan skor untuk setiap dokumen
    scores = bm25.get_scores(tokenized_query)

    # Gabungkan skor dengan indeks dan urutkan
    scored_results = [(score, idx) for idx, score in enumerate(scores)]
    scored_results.sort(reverse=True) # Mengurutkan data dari yang tertinggi

    top_results = []
    for i in range(min(top_n, len(scored_results))):
        if scored_results[i][0] > 0:
            top_results.append(scored_results[i])

    return top_results

7. Meminta pengguna untuk memasukkan kata kunci pencarian melalui command line dan menyimpan input tersebut dalam variabel query.



In [ ]:
# 7. Minta input query
query = input("Masukkan kata kunci pencarian judul film: ")

# gunakan fungsi tadi untuk mendapatkan judul
top_results = search_movies(query)
print(f"\nHasil pencarian untuk: '{query}'\n")

# tampilakn hasil dari pencarian
if len(top_results) == 0:
    print("Tidak ditemukan hasil yang cocok dengan pencarian Anda.")
else:
    for i, (score, idx) in enumerate(top_results, 1):
        movie = df.iloc[idx]
        relevance_percent = score  # Konversi skor ke persentase relevansi (dengan batas 100%)

        print(f"{i}. {movie['title']} - Score BM25: {relevance_percent}")
        print(f"   Tahun: {movie['release_date'][:4]}")
        print(f"   Rating: {movie['vote_average']} ({movie['vote_count']} votes)")
        print(f"   Overview: {movie['overview'][:150]}..." if len(movie['overview']) > 150 else movie['overview'])
        print()

Masukkan kata kunci pencarian judul film: spiderman

Hasil pencarian untuk: 'spiderman'

1. Spider-Man - Score BM25: 9.532889440786716
   Tahun: 2002
   Rating: 7.309 (19538 votes)
   Overview: After being bitten by a genetically altered spider at Oscorp, nerdy but endearing high school student Peter Parker is endowed with amazing powers to b...

2. Spider-Man - Score BM25: 9.532889440786716
   Tahun: 2002
   Rating: 7.3 (19538 votes)
   Overview: After being bitten by a genetically altered spider at Oscorp, nerdy but endearing high school student Peter Parker is endowed with amazing powers to b...

3. Spider-Man 3 - Score BM25: 7.780738361426088
   Tahun: 2007
   Rating: 6.435 (14410 votes)
   Overview: The seemingly invincible Spider-Man goes up against an all-new crop of villains—including the shape-shifting Sandman. While Spider-Man’s superpowers a...

4. Spider-Man 2 - Score BM25: 7.780738361426088
   Tahun: 2004
   Rating: 7.3 (15437 votes)
   Overview: Peter Parker is going thro